In [1]:
import pandas as pd

## State-by-state summaries

Import New England summary data (Vermont, Maine, New Hampshire, Massachusetts, Rhode Island and Connecticut) for all three years, add a "year" column and combine all; drop columns not pertinent to opioid prescriptions.

In [2]:
ne2013 = pd.read_csv("data/NewEnglandPrescriberSummary2013.csv")
ne2013 = ne2013.rename(columns={
    'nppes_provider_state': 'state'
})


ne2014 = pd.read_csv("data/NewEnglandPrescriberSummary2014.csv")
ne2014 = ne2014.rename(columns={
    'nppes_provider_state': 'state'
})


ne2015 = pd.read_csv("data/NewEnglandPrescriberSummary2015.csv")
ne2015 = ne2015.rename(columns={
    'nppes_provider_state': 'state'
})

ne2013['year'] = 2013
ne2014['year'] = 2014
ne2015['year'] = 2015

ne = pd.concat([ne2013,ne2014,ne2015])
ne = ne.drop(['nppes_credentials','nppes_provider_street1','nppes_provider_street2',
                                'nppes_provider_zip5','total_drug_cost_ge65',
                                'total_30_day_fill_count_ge65','total_day_supply_ge65','ge65_suppress_flag',
                                'total_claim_count_ge65','antibiotic_claim_count','antibiotic_drug_cost',
                                'antibiotic_bene_count','nppes_provider_gender','nppes_entity_code',
                                'nppes_provider_zip4','nppes_provider_country','description_flag',
                                'medicare_prvdr_enroll_status','total_30_day_fill_count','total_day_supply',
                                'total_drug_cost','bene_count_ge65_suppress_flag','bene_count_ge65',
                                'beneficiary_race_white_count','beneficiary_race_black_count',
                                'brand_suppress_flag','brand_claim_count','beneficiary_female_count',
                                'beneficiary_male_count','beneficiary_race_asian_pi_count','nonlis_claim_count',
                                'beneficiary_race_hispanic_count','beneficiary_race_nat_ind_count',
                                'brand_drug_cost','generic_suppress_flag','generic_drug_cost','hrm_ge65_suppress_flag',
                                'generic_claim_count','beneficiary_race_other_count','beneficiary_nondual_count',
                                'beneficiary_dual_count','beneficiary_average_risk_score','other_suppress_flag',
                                'other_claim_count','antipsych_claim_count_ge65','antipsych_drug_cost_ge65',
                                'other_drug_cost','mapd_suppress_flag','hrm_bene_count_ge65','mapd_claim_count',
                                'mapd_drug_cost','hrm_bene_ge65_suppress_flag','pdp_suppress_flag','pdp_claim_count',
                                'pdp_drug_cost','lis_suppress_flag','hrm_drug_cost_ge65','lis_claim_count',
                                'antipsych_ge65_suppress_flag','antipsych_bene_ge65_suppress_flg','lis_drug_cost',
                                'antipsych_bene_count_ge65','antipsych_bene_count_ge65','nonlis_suppress_flag',
                                'beneficiary_age_less_65_count','beneficiary_age_65_74_count','nonlis_drug_cost',
                                'hrm_claim_count_ge65','beneficiary_age_75_84_count',
                                'beneficiary_age_greater_84_count', 'opioid_drug_cost', 'nppes_provider_mi',
                                'total_claim_count','bene_count','opioid_prescriber_rate',
                                'average_age_of_beneficiaries','nppes_provider_city','nppes_provider_last_org_name',
                                'nppes_provider_first_name','specialty_description'
                               ],axis=1).rename(columns={'nppes_provider_state': 'state'})

ne.head()

,npi,state,opioid_claim_count,opioid_day_supply,opioid_bene_count,year
0,1649426404,ME,0.0,0.0,0.0,2013
1,1952310666,CT,NaN,NaN,NaN,2013
2,1780778795,CT,0.0,0.0,0.0,2013
3,1669400222,ME,51.0,716.0,NaN,2013
4,1417246950,MA,NaN,NaN,NaN,2013


Fill in "5" for rows where beneficiary count is null, but drop rows where day supply and claim count are also null:

In [3]:
ne_fillna = ne.copy()
ne_fillna.opioid_bene_count.fillna(5, inplace=True)

ne_fillna = ne_fillna[pd.notnull(ne_fillna['opioid_day_supply'])]

ne_fillna.head()

,npi,state,opioid_claim_count,opioid_day_supply,opioid_bene_count,year
0,1649426404,ME,0.0,0.0,0.0,2013
2,1780778795,CT,0.0,0.0,0.0,2013
3,1669400222,ME,51.0,716.0,5.0,2013
5,1518184605,RI,124.0,1344.0,55.0,2013
8,1962694323,MA,0.0,0.0,0.0,2013


How large was the initial dataset?

In [4]:
len(ne)

205764

How many did we lose by dropping rows with multiple blanks?

In [5]:
100.0*(len(ne) - len(ne_fillna))/len(ne)

31.88701619330884

About 32 percent. Not ideal, but there's no way around it. Let's summarize it by state:

In [6]:
nesummary = ne_fillna.groupby(['year','state'], as_index=False).sum()
nesummary = nesummary.drop(['npi'], axis=1)

## Prescription data summaries

In [7]:
prescrip2013 = pd.read_csv("data/Vermont_Prescriptions_By_Prescriber_2013.csv",
                          dtype={'bene_count':'float64',
                                 'total_claim_count': 'float64',
                                 'total_day_supply':'float64'
                                })

prescrip2014 = pd.read_csv("data/Vermont_Prescriptions_By_Prescriber_2014.csv",
                           dtype={'bene_count':'float64',
                                 'total_claim_count': 'float64',
                                 'total_day_supply':'float64'
                                })

prescrip2015 = pd.read_csv("data/Vermont_Prescriptions_By_Prescriber_2015.csv",
                          dtype={'bene_count':'float64',
                                 'total_claim_count': 'float64',
                                 'total_day_supply':'float64'
                                })

prescrip2013['year'] = 2013
prescrip2014['year'] = 2014
prescrip2015['year'] = 2015

prescrip = pd.concat([prescrip2013, prescrip2014, prescrip2015])
prescrip = prescrip.drop(['nppes_provider_state','description_flag','bene_count_ge65',
                                  'bene_count_ge65_suppress_flag','ge65_suppress_flag','total_30_day_fill_count_ge65',
                                  'total_day_supply_ge65','total_drug_cost_ge65','total_claim_count_ge65'], axis=1)

prescrip = prescrip.rename(columns={
    'nppes_provider_last_org_name': 'last_name',
    'nppes_provider_first_name': 'first_name',
    'nppes_provider_city':'city',
    'specialty_description': 'specialty'
})

prescrip.head()

,npi,last_name,first_name,city,specialty,drug_name,generic_name,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,year
0,1669404422,FOOTE,PETER,BRATTLEBORO,Family Practice,AZITHROMYCIN,AZITHROMYCIN,17.0,19.0,19.0,95.0,$180.65,2013
1,1073634531,GILBERT,MATTHEW,SOUTH BURLINGTON,Endocrinology,AMLODIPINE BESYLATE,AMLODIPINE BESYLATE,NaN,11.0,19.0,570.0,$83.74,2013
2,1992791172,UITERWYK,SEAN,WHITE RIVER JUNCTION,Family Practice,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,NaN,22.0,23.0,516.0,$237.92,2013
3,1497718928,ECKHAUS,JEREMIAN,MONTPELIER,Family Practice,CLOPIDOGREL,CLOPIDOGREL BISULFATE,NaN,18.0,26.0,746.0,$562.23,2013
4,1043220056,NOVAS-SCHMIDT,MARIA,BURLINGTON,Psychiatry,LITHIUM CARBONATE,LITHIUM CARBONATE,NaN,18.0,18.0,540.0,$351.15,2013


Merge a list of generic opioid names with the larger dataset. Medication names are coded for type (using CDC and FDA sources):

* M: Maintenance/treatment opioid — methadone and buprenorphine
* A: Most abused opioid — hydrocodone, oxycodone, fentanyl, morphine & hydromorphone
* O: All other opioid drugs

Then fill all blank bene_count rows with middle value of 5.

In [8]:
opioids = pd.read_csv('data/opioid-names.csv')

opioidprescrip = prescrip.merge(opioids, how='inner', right_on='medication', left_on='generic_name')
opioidprescrip.bene_count.fillna(5, inplace=True)
opioidprescrip = opioidprescrip.drop(['npi','total_30_day_fill_count', 'medication','total_drug_cost'], axis=1)
opioidprescrip.head()

,last_name,first_name,city,specialty,drug_name,generic_name,bene_count,total_claim_count,total_day_supply,year,type
0,UITERWYK,SEAN,WHITE RIVER JUNCTION,Family Practice,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,5.0,22.0,516.0,2013,O
1,BEATTY,DENNIS,SOUTH BURLINGTON,Internal Medicine,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,5.0,12.0,167.0,2013,O
2,STALL,JEFFREY,BOMOSEEN,Internal Medicine,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,5.0,14.0,285.0,2013,O
3,SANDOVAL,MARIE,SOUTH BURLINGTON,Internal Medicine,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,5.0,11.0,277.0,2013,O
4,SARGENT,BRIAN,RANDOLPH,Family Practice,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,5.0,12.0,276.0,2013,O


Summarize all drugs by year, generic name and type.

In [10]:
opioidsummary = opioidprescrip.groupby(['year','generic_name','type'], as_index=False).sum()

opioidsummary.head()

,year,generic_name,type,bene_count,total_claim_count,total_day_supply
0,2013,ACETAMINOPHEN WITH CODEINE,O,992.0,2617.0,41697.0
1,2013,BUPRENORPHINE,M,19.0,61.0,1366.0
2,2013,BUPRENORPHINE HCL,M,70.0,540.0,6374.0
3,2013,BUPRENORPHINE HCL/NALOXONE HCL,M,628.0,8030.0,98887.0
4,2013,BUTALBIT/ACETAMIN/CAFF/CODEINE,O,5.0,14.0,378.0
